<a href="https://colab.research.google.com/github/Jwilson1172/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import numpy as np
import pandas as pd
from scipy import stats

# column headers
names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

# init the datra frame from csv
df=pd.read_csv('/content/house-votes-84.data',names=names)


In [5]:
# make sure that the changes persisted
df=df.replace({'n':0,'y':1,'?':np.NaN})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [8]:
rep=df[df['party']=='republican']
dem=df[df['party']=='democrat']

# using ugly output to print heads to see if it looks right
print(rep.head())
print(dem.head())


         party  handicapped-infants  ...  duty-free  south-africa
0   republican                  0.0  ...        0.0           1.0
1   republican                  0.0  ...        0.0           NaN
7   republican                  0.0  ...        NaN           1.0
8   republican                  0.0  ...        0.0           1.0
10  republican                  0.0  ...        0.0           0.0

[5 rows x 17 columns]
      party  handicapped-infants  water-project  ...  crime  duty-free  south-africa
2  democrat                  NaN            1.0  ...    1.0        0.0           0.0
3  democrat                  0.0            1.0  ...    0.0        0.0           1.0
4  democrat                  1.0            1.0  ...    1.0        1.0           1.0
5  democrat                  0.0            1.0  ...    1.0        1.0           1.0
6  democrat                  0.0            1.0  ...    1.0        1.0           1.0

[5 rows x 17 columns]


In [19]:
# find the mean values for each column in both data sets
# this is going to tell me what issues each party voted for
# excluding party column because it dosent make sense to take the mean of a string
print('\nREP:')
for col in names[1:]:
  print(f"issue: {col} Mean votes: {rep[col].describe()['mean']}")
print("\nDEM:")
for col in names[1:]:
  print(f"issue: {col} Mean votes: {dem[col].describe()['mean']}")



REP:
issue: handicapped-infants Mean votes: 0.18787878787878787
issue: water-project Mean votes: 0.5067567567567568
issue: budget Mean votes: 0.13414634146341464
issue: physician-fee-freeze Mean votes: 0.9878787878787879
issue: el-salvador-aid Mean votes: 0.9515151515151515
issue: religious-groups Mean votes: 0.8975903614457831
issue: anti-satellite-ban Mean votes: 0.24074074074074073
issue: aid-to-contras Mean votes: 0.15286624203821655
issue: mx-missile Mean votes: 0.11515151515151516
issue: immigration Mean votes: 0.5575757575757576
issue: synfuels Mean votes: 0.1320754716981132
issue: education Mean votes: 0.8709677419354839
issue: right-to-sue Mean votes: 0.8607594936708861
issue: crime Mean votes: 0.9813664596273292
issue: duty-free Mean votes: 0.08974358974358974
issue: south-africa Mean votes: 0.6575342465753424

DEM:
issue: handicapped-infants Mean votes: 0.6046511627906976
issue: water-project Mean votes: 0.502092050209205
issue: budget Mean votes: 0.8884615384615384
issue: 

# Finding a Republican issue

###Null Hypothesis: 
Republican support for the 'crime' bill ***is not*** equal to the democrats support for the 'crime' bill.

$ H_0: \bar{x}_{r} \neq \bar{x}_{d}$

###Alternate Hypothesis: 
Republican support for the bill 'crime' is equal to the democrates support for the 'crime' bill

$ H_a: \bar{x}_{r} == \bar{x}_{d}$

###Confidence Level: 
95% (My critical value is .05) -> (1-.95 == .05)


In [20]:
# ttest if the republicans and democarts support for the'crime' bill differs
stats.ttest_ind(rep['crime'],dem['crime'],nan_policy='omit')

Ttest_indResult(statistic=16.342085656197696, pvalue=9.952342705606092e-47)

##Conclusion:
bases on the t-stat of 16.34 and the p-value of 9.95x10<sup>-47</sup> I <b>fail to reject</b> the null hypothesis that the support for the 'crime' bill is equal between Democrats and Republicans

#Finding A Democrat Issue:

###Null Hypothesis: 
Republican support for the 'budget' bill ***is not*** equal to the democrats support for the 'budget' bill.

$ H_0: \bar{x}_{r} \neq \bar{x}_{d}$

###Alternate Hypothesis: 
Republican support for the bill 'budget' is equal to the democrates support for the 'budget' bill

$ H_a: \bar{x}_{r} == \bar{x}_{d}$

###Confidence Level: 
95% (My critical value is .05) -> (1-.95 == .05)

In [21]:
# ttest if the democrats and repbulicans have diffreing views on the 'budget' bill
stats.ttest_ind(rep['budget'],dem['budget'],nan_policy='omit')

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

##Conclusion:
bases on the t-stat of -23.21 and the p-value of 2.07x10<sup>-77</sup> I <b>fail to reject</b> the null hypothesis that the support for the 'budget' bill is equal between Democrats and Republicans

#Finding A Bi-Partisan Issue:

###Null Hypothesis: 
Republican support for the 'water-project' bill ***is not*** equal to the democrats support for the 'water-project' bill.

$ H_0: \bar{x}_{r} \neq \bar{x}_{d}$

###Alternate Hypothesis: 
Republican support for the bill 'water-project' is equal to the democrates support for the 'water-project' bill

$ H_a: \bar{x}_{r} == \bar{x}_{d}$

###Confidence Level: 
95% (My critical value is .05) -> (1-.95 == .05)

In [23]:
stats.ttest_ind(rep['water-project'],dem['water-project'],nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

##Conclusion:
bases on the t-stat of 0.089 and the p-value of ~1 I <b>Reject</b> the null hypothesis that the support for the 'water-project' bill is not equal between Democrats and Republicans, and sujgest the alternitive hypothesis, the the support for the water bill is closer to 50/50

##Stretch Goal:

In [0]:
# making a funtion to find abbitrary inputs pythonicly
def ttest_ind_all(df1:pd.DataFrame,df2:pd.DataFrame,columns="")->type(None):
  '''
    A function that takes 2 data frames and conducts ttest on all of the columns in thoose dataframes
    
    @param df1: the first data frame to test, NOTE: this is the df that has its columns indexed
    @param df2: the second dataframe to test the frist data frame against
    @columns: specific columns to test in the two dataframes NOTE: please don't df.columns this arg
    it wasnt ment for that it was ment for a subset of df.columns that doesn't include the 'name' column
  '''
  # if there are no columns passed
  if columns=="":
    # i want to itterat throught the columns but not 'name'
    # @todo implement more arguments to switch offset
    for col in df1.columns.to_numpy()[1:]:
      # @todo implement more arguments to switch nan_policy
      tt=stats.ttest_ind(df1[col],df2[col],nan_policy='omit')
      #print output of ttest to console
      print(f"The results of ttest_ind for Column:{col} are,\n{tt}")
  else:
    # I'm assuming a 'smart' user that passes a np.array with ONLY the columns to test
    for col in columns:
      # @todo implement more arguments to switch nan_policy
      tt=stats.ttest_ind(df1[col],df2[col],nan_policy='omit')
      #print output of ttest to console
      #@todo clear garbage out of output so it is an easier read
      print(f"The results of ttest_ind for Column:{col} are,\n{tt}")
  return 

In [27]:
# hmmm tasty type garbage, soo paletable to a human
ttest_ind_all(rep,dem,columns=['budget','water-project'])

The results of ttest_ind for Column:budget are,
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
The results of ttest_ind for Column:water-project are,
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)


In [28]:
# looks like both versions worked
ttest_ind_all(rep,dem)

The results of ttest_ind for Column:handicapped-infants are,
Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
The results of ttest_ind for Column:water-project are,
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
The results of ttest_ind for Column:budget are,
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
The results of ttest_ind for Column:physician-fee-freeze are,
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
The results of ttest_ind for Column:el-salvador-aid are,
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
The results of ttest_ind for Column:religious-groups are,
Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
The results of ttest_ind for Column:anti-satellite-ban are,
Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)
The results of ttest_ind for Column:aid-to-contras are,
Ttest_indRes